In [1]:
import numpy as np
import pandas as pd
import recommenders as rec
import networkx as nx
import matplotlib.pyplot as plt
import areamanager
import math
import timeit
import geo_utils
from constants import geocat_constants
from constants import experiment_constants
from constants import usg_constants
from collections import defaultdict
import utils
def string_to_array(string):
    if string == '':
        return list()
    return eval(string)
%matplotlib inline

from lib.UserBasedCF import UserBasedCF
from lib.FriendBasedCF import FriendBasedCF
from lib.PowerLaw import PowerLaw

from lib.metrics import precisionk, recallk, ildk,gck

# poi_int_id=df_checkin_train['business_id'].drop_duplicates().reset_index(drop=True).to_dict()
# poi_num=len(poi_int_to_id)
# user_int_id=df_checkin_train.reset_index().user_id.drop_duplicates().reset_index(drop=True).to_dict()
# user_num=len(user_int_to_id)
# for i,j in poi_int_id.copy().items():
#     poi_int_id[j]=i
# for i,j in user_int_id.copy().items():
#     user_int_id[j]=i
# def user_id_to_int(user_id):
#     return user_int_id[user_id]
# def poi_id_to_int(user_id):
#     return poi_int_id[user_id]

CITY=experiment_constants.get_city()

In [2]:
df_checkin=pd.read_csv("../data/checkin/"+CITY+".csv",converters={'categories':string_to_array})

df_checkin_train=pd.read_csv("../data/checkin/train/"+CITY+".csv",converters={'categories':string_to_array})


users_id=df_checkin['user_id'].drop_duplicates().reset_index(drop=True)
pois_id=df_checkin['business_id'].drop_duplicates().reset_index(drop=True)
user_num=len(users_id)
poi_num=len(pois_id)
# USG uses unique checkins cij = 1 or cij = 0
#df_checkin_train=df_checkin_train.drop_duplicates(subset=['user_id','business_id']).reset_index(drop=True)

df_checkin_train=df_checkin_train.set_index('user_id')
df_checkin_train.head()

,business_id,date,latitude,longitude,categories,subarea_id
user_id,,,,,,
0,1744,2017-12-15 14:05:01,43.075012,-89.447498,"[desserts, cafes, coffee]",2143.0
0,2746,2018-01-18 20:57:23,43.106774,-89.497738,"[desserts, tradamerican, diners, gastropubs, b...",2641.0
0,30,2018-01-23 23:14:37,43.072654,-89.384062,"[bars, mexican]",2133.0
0,432,2018-01-27 02:17:06,43.072439,-89.384610,"[tradamerican, bars]",2133.0
0,22,2018-01-27 02:31:26,43.075100,-89.381069,[mexican],2132.0


In [3]:
df_checkin_test=pd.read_csv("../data/checkin/test/"+CITY+".csv",converters={'categories':string_to_array})


In [4]:
ground_truth = defaultdict(set)
for index,row in df_checkin_test[['user_id','business_id']].drop_duplicates().iterrows():
    ground_truth[row['user_id']].add(row['business_id'])

In [5]:
df_user_friends=pd.read_csv("../data/user/"+CITY+".csv",converters={'friends':string_to_array})

In [6]:
poi_coos = {}
for lid,latlon in df_checkin[['business_id','latitude','longitude']].reset_index(drop=True).drop_duplicates().set_index('business_id').iterrows():
    poi_coos[lid] = tuple(latlon)

In [7]:
poi_cats = {}
for lid,row in df_checkin[['business_id','categories']].reset_index(drop=True).drop_duplicates(subset=['business_id']).set_index('business_id').iterrows():
    poi_cats[lid] = row.categories


In [8]:
import geocat
dict_alias_title,category_tree,dict_alias_depth=geocat.category_manipulation_utils()
undirected_category_tree=category_tree.to_undirected()

In [9]:
training_matrix = np.zeros((user_num, poi_num))


In [10]:
for user_id,poi_id in df_checkin_train.business_id.iteritems():
   # print(user_id,poi_id)
    training_matrix[user_id,poi_id]=1

In [11]:
social_relations = defaultdict(list)
for user_id,friends in df_user_friends.set_index("user_id").friends.iteritems():
    social_relations[user_int_id[user_id]]=[user_int_id[uid] for uid in friends]

In [12]:
top_k = 100
alpha = 0.1
beta = 0.1

U = UserBasedCF()
S = FriendBasedCF(eta=0.05)
G = PowerLaw()

U.pre_compute_rec_scores(training_matrix)
S.compute_friend_sim(social_relations, training_matrix)
G.fit_distance_distribution(training_matrix, poi_coos)

Training User-based Collaborative Filtering...
Done. Elapsed time: 0.8136675357818604 s
Precomputing similarity between friends...
Done. Elapsed time: 0.0005125999450683594 s
fitting distance distribution...
done. elapsed time: 2.500680446624756 s


In [13]:
result_out = open("../data/result/sigir11_top_" + str(top_k) + ".txt", 'w')

all_uids = list(range(user_num))
all_lids = list(range(poi_num))



In [14]:
np.random.shuffle(all_uids)

In [15]:
precision, recall = [], []

In [16]:
def normalize(scores):
    max_score = max(scores)
    if not max_score == 0:
        scores = [s / max_score for s in scores]
    return scores


In [17]:
ild, gc = [],[]

In [18]:
# df_poi_neighbor=pd.read_csv("../data/neighbor/"+CITY+".csv",converters={'neighbors':string_to_array})
# df_poi_neighbor=df_poi_neighbor.set_index("business_id")

In [19]:
# poi_neighbors={}
# for lid,row in df_poi_neighbor.iterrows():
#     neighbors=row['neighbors']
#     poi_neighbors[lid]=neighbors

In [20]:
#log_size=training_matrix[0,training_matrix[0,:].nonzero()[0]].sum()

In [21]:
training_matrix[0,:].nonzero()[0]

array([   0,   22,   30,   38,  299,  346,  415,  432,  945, 1036, 1079,
       1168, 1213, 1470, 1671, 1744, 1852, 2038, 2051, 2317, 2600, 2734,
       2746, 3086])

In [25]:
for cnt, uid in enumerate(all_uids):
    if uid in ground_truth:
        U_scores = normalize([U.predict(uid, lid)
                              if training_matrix[uid, lid] == 0 else -1
                              for lid in all_lids])
        S_scores = normalize([S.predict(uid, lid)
                              if training_matrix[uid, lid] == 0 else -1
                              for lid in all_lids])
        G_scores = normalize([G.predict(uid, lid)
                              if training_matrix[uid, lid] == 0 else -1
                              for lid in all_lids])
        
        U_scores = np.array(U_scores)
        S_scores = np.array(S_scores)
        G_scores = np.array(G_scores)
        
        overall_scores = (1.0 - alpha - beta) * U_scores + alpha * S_scores + beta * G_scores
        
        predicted = list(reversed(overall_scores.argsort()))[:top_k]
        actual = ground_truth[uid]
        
        precision.append(precisionk(actual, predicted[:10]))
        recall.append(recallk(actual, predicted[:10]))

        
        
        ild.append(ildk(predicted[:10],poi_cats,undirected_category_tree))
        gc.append(gck(uid,training_matrix,poi_cats,predicted[:10]))
        print(cnt, uid, "pre@10:", np.mean(precision), "rec@10:", np.mean(recall),"ild@10:",np.mean(ild),"gc@10:",np.mean(gc))
        result_out.write(str(uid)+',"'+str(predicted)+'"\n')
result_out.close()

4.715236400634448e-12
0 69 pre@10: 0.06857142857142857 rec@10: 0.0368612971639606 ild@10: 0.3271111111111112 gc@10: 0.45081594575104567
0.004638003386817181
1 59 pre@10: 0.06944444444444446 rec@10: 0.03625824766361489 ild@10: 0.32748971193415644 gc@10: 0.4463951324431463
0.002550894336489344
2 393 pre@10: 0.07567567567567568 rec@10: 0.039332349078111783 ild@10: 0.3278478478478479 gc@10: 0.44559166039513337
0.005405465169173986
3 627 pre@10: 0.0736842105263158 rec@10: 0.03829728726026674 ild@10: 0.3289668615984406 gc@10: 0.44601131305679986
3.060518890261289e-09
4 106 pre@10: 0.07435897435897437 rec@10: 0.04052043374077272 ild@10: 0.3281291547958215 gc@10: 0.4459711369385343
0.001892457072198969
5 515 pre@10: 0.07250000000000001 rec@10: 0.03950742289725341 ild@10: 0.32992592592592607 gc@10: 0.44419685851507096
0.0025466787022591793
6 596 pre@10: 0.07073170731707318 rec@10: 0.038543827216832596 ild@10: 0.3298283649503163 gc@10: 0.4447449026163293
8.921662688916664e-12
7 1021 pre@10: 0.06

KeyboardInterrupt: 

In [ ]:
df_metrics=pd.DataFrame([ild,gc]).T
df_metrics.columns=['ild','gc']

In [ ]:
df_metrics=df_metrics.sort_values('ild')

df_metrics=df_metrics.reset_index(drop=True)

df_metrics.plot()

In [ ]:
df_metrics.corr()

In [ ]:
### df_metrics=df_metrics.sort_values('gc')

df_metrics=df_metrics.reset_index(drop=True)

df_metrics.plot()